In [ ]:
# Instal necessary libraies
!pip install -U spacy
!pip install -U pip setuptools wheel

In [ ]:
# Check spacy version
!pip show spacy

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# Load spacy langugage model
!python -m spacy download en_core_web_trf

In [ ]:
# Load spacy relation extraction dependecies
!python -m spacy project clone tutorials/rel_component

In [ ]:
# Load spacy transfomers
!pip install -U spacy transformers

In [ ]:
# Change Directory
%cd /content/rel_component

In [ ]:
# Train models
!spacy project run train_gpu
!spacy project run train_cpu

In [ ]:
# Evaluate the trained model
!spacy project run evaluate

#### Relation extraction
- Based on Named Entities and labeled relations

In [ ]:
# Import necessary libraries
import spacy
from spacy import util

In [ ]:
# Load Trained NER Model (Transformer_Based_NER)
nlp = spacy.load("/content/drive/MyDrive/Datasets/TheArchitect/model-best")

In [ ]:
text = [
'''EG is an energy provider who allows their customers to access the service using web browsers and also from mobile devices.
   The online service will allow users to login to the system and submit the electricity and gas meter readings.
   EG will notify users on successful submission of meter readings also sends the bill amount based on the calculation.
'''
]

In [ ]:
for doc in nlp.pipe(text, disable=["tagger"]):
   print(f"spans: {[(e.start, e.text, e.label_) for e in doc.ents]}")

In [ ]:
import random
import typer
from pathlib import Path
import spacy
from spacy.tokens import DocBin, Doc
from spacy.training.example import Example

from rel_pipe import make_relation_extractor, score_relations
from rel_model import create_relation_model, create_classification_layer, create_instances, create_tensors

In [ ]:
# Load trained relation extraction model
nlp_relations = spacy.load("training/model-best")
nlp_relations.add_pipe("sentencizer")

In [ ]:
for name, proc in nlp2.pipeline:
          doc = proc(doc)

In [ ]:
# Here, we split the paragraph into sentences and apply the relation extraction for each pair of entities found in each sentence.
for value, rel_dict in doc._.rel.items():
        for sent in doc.sents:
          for e in sent.ents:
            for b in sent.ents:
              if e.start == value[0] and b.start == value[1]:
                if rel_dict['ACCESS_USING'] >=0.8 :
                  print(f" entities: {e.text, b.text} --> predicted relation: {rel_dict}")

In [ ]:
for value, rel_dict in doc._.rel.items():
    for sent in doc.sents:
        for e in sent.ents:
            for b in sent.ents:
                if e.start == value[0] and b.start == value[1]:
                    max_rel = max(rel_dict, key=rel_dict.get) # Get the key with the maximum value
                    print(f"entities: {e.text, b.text} --> predicted relation: {max_rel} (score: {rel_dict[max_rel]})")


In [ ]:
import json

architecture_data = {
    "actors": [],
    "containers": [],
    "databases": [],
    "systems": [],
    "relationships": []
}

unique_entities = set()
entity_relationships = []

# Collect unique entities and their relationships
for value, rel_dict in doc._.rel.items():
    for sent in doc.sents:
        for e in sent.ents:
            for b in sent.ents:
                if e.start == value[0] and b.start == value[1]:
                    max_rel = max(rel_dict, key=rel_dict.get) # Get the key with the maximum value
                    unique_entities.add(e.text)
                    unique_entities.add(b.text)
                    entity_relationships.append((e.text, b.text, max_rel, rel_dict[max_rel]))

# Add unique entities to the appropriate list in the dictionary
for entity in unique_entities:
    # Customize this part to match your entity categorization logic
    if entity.lower() in {"energy provider", "customers", "users"}:
        architecture_data["actors"].append({"name": entity, "description": ""})
    elif entity.lower() in {"eg", "online service", "login"}:
        architecture_data["containers"].append({"name": entity, "technology": "", "description": ""})
    elif entity.lower() in {"mobile devices", "web browsers"}:
        architecture_data["systems"].append({"name": entity, "description": "", "external": True})

# Add relationships to the dictionary
for e1, e2, rel, score in entity_relationships:
    architecture_data["relationships"].append({
        "source": e1,
        "relationship": rel,
        "target": e2,
        "score": score
    })

# Convert the dictionary to a JSON object
architecture_json = json.dumps(architecture_data, indent=2)
print(architecture_json)
